## Number Detector

### Problem Statement

Given a set of images of football players, identify their jersey numbers from the images. The images are taken from various angles. The goal is to develop a model with a high accuracy that can detect a jersey number from an image taken from any angle.

### Dataset
The main dataset I used is from here: https://www.kaggle.com/datasets/frlemarchand/nfl-player-numbers/data

The full dataset contains about 43,500 64x64 images of NFL players with jersey numbers. The dataset appears to have some images where it is difficult, even for a human, to identify a player number. The images appear to come from All-22 film, with images from both the sideline and endzone view. 

I intend to use a randomly selected subset of images, as it is not feasible to upload all of these images to GitHub, and training a model on all of these images would be computationally expensive.

### To Do List
- Reduce size of dataset
- Read csv file
- Obtain labels from csv file, based on the images being used
- Preprocess images
- Create a model
- Train the model
- Test the model
- Evaluate the model

### Reducing Full Dataset

In [2]:
import os
import random

In [8]:
data_dir = os.path.join('archive', 'train_player_numbers')
image_count = 43540

def select_images(data_dir, sample_size = 1000, extensions = ('.jpg', '.jpeg', '.png')):
    image_files = [file for file in os.listdir(data_dir) if file.lower().endswith(extensions)]
    img_sample = random.sample(image_files, sample_size)
    
    for image in image_files:
        if image not in img_sample:
            os.remove(os.path.join(data_dir, image))

In [9]:
select_images(data_dir)
    

In [11]:
image_files = [file for file in os.listdir(data_dir)]
len(image_files)

1000